In [10]:
#firstly, extracting all items inside a user's iventory (only cs2 items)
import requests
import time
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/117.0.0.0 Safari/537.36"
}
def fetch_inventory_page(steamid64: str, appid: int, contextid: int, start: str = None):
    url = f"https://steamcommunity.com/inventory/{steamid64}/{appid}/{contextid}"
    resp = requests.get(url,headers=HEADERS, timeout=15)
    resp.raise_for_status()
    return resp.json()
def inventory_items_for_user(steamid64: str, appid: int, contextid: int, delay_between_requests: float = 0.5):
    items = []
    seen_assetids = set()
    start = None

    while True:
        data = fetch_inventory_page(steamid64, appid, contextid, start)
        # check if inventory is private or empty
        if not data or "assets" not in data or "descriptions" not in data:
            # Might be private or zero items; return whatever we have or raise
            return items, data

        assets = data.get("assets", [])
        descriptions = data.get("descriptions", [])

        # Build lookup for descriptions by (classid, instanceid)
        desc_lookup = {}
        for d in descriptions:
            key = (str(d.get("classid")), str(d.get("instanceid", "0")))
            desc_lookup[key] = d

        for a in assets:
            assetid = a.get("assetid")  # unique per item
            if assetid in seen_assetids:
                continue
            seen_assetids.add(assetid)

            classid = str(a.get("classid"))
            instanceid = str(a.get("instanceid", "0"))
            desc = desc_lookup.get((classid, instanceid)) or {}

            item = {
                "assetid": assetid,
                "classid": classid,
                "instanceid": instanceid,
                "amount": a.get("amount", 1),
                "market_tradable_restriction": a.get("market_tradable_restriction"),
                # descriptive fields:
                "name": desc.get("name"),
                "type": desc.get("type"),
                "market_hash_name": desc.get("market_hash_name"),
                "tradable": desc.get("tradable"),
                "tags": desc.get("tags", []),
                "marketable": desc.get("marketable"),
                # entire desc for debugging
                "description_raw": desc
            }
            items.append(item)

        # pagination
        more = data.get("more_items") or False
        # some responses give 'last_assetid' to use as start; others use 'more_start'
        if more:
            start = data.get("last_assetid") or data.get("more_start")
            if not start:
                # fallback: if server indicates 'more_items' but no token, stop to avoid infinite loop
                break
            time.sleep(delay_between_requests)  # be polite
        else:
            break

    return items, data

In [ ]:
if __name__ == "__main__":
    # Replace these with real values
    steamid64 = ""   # example Ste"   # example SteamID64
    appid = 730                       # likely appid for CS:GO/CS2 (confirm)
    contextid = 2                     # common contextid; confirm for your target game

    items, raw = inventory_items_for_user(steamid64, appid, contextid)
    print(f"Found {len(items)} items")
    for i, it in enumerate(items[:200], start=1):
        print(f"{i}. {it['name']!r} {it['market_hash_name']!r} ({it['assetid']}) - tradable={it.get('tradable')} marketable={it.get('marketable')}")

Found 59 items
1. 'Sticker | NRG | Austin 2025' 'Sticker | NRG | Austin 2025' (46497322311) - tradable=1 marketable=1
2. 'Sticker | reck (Foil) | Austin 2025' 'Sticker | reck (Foil) | Austin 2025' (46497308493) - tradable=1 marketable=1
3. 'Sticker | broky (Foil) | Austin 2025' 'Sticker | broky (Foil) | Austin 2025' (46497303228) - tradable=1 marketable=1
4. 'Sticker | skullz (Foil) | Austin 2025' 'Sticker | skullz (Foil) | Austin 2025' (46497299723) - tradable=1 marketable=1
5. 'Sticker | yuurih (Foil) | Austin 2025' 'Sticker | yuurih (Foil) | Austin 2025' (46497297031) - tradable=1 marketable=1
6. 'Sticker | m0NESY (Foil) | Austin 2025' 'Sticker | m0NESY (Foil) | Austin 2025' (46497275563) - tradable=1 marketable=1
7. 'Sticker | jottAAA (Foil) | Austin 2025' 'Sticker | jottAAA (Foil) | Austin 2025' (46497270309) - tradable=1 marketable=1
8. 'Sticker | Heavygod (Gold) | Austin 2025' 'Sticker | Heavygod (Gold) | Austin 2025' (46497253994) - tradable=1 marketable=1
9. 'FAMAS | Grey Ghos

In [17]:
#now, use the name of items, and fetch the current market price
from urllib.parse import quote
def get_market_price(market_hash_name, appid=730, currency=1):
    base_url = "https://steamcommunity.com/market/priceoverview/"
    params = {
        "appid": appid,
        "currency": currency,
        "market_hash_name": market_hash_name
    }
    url = base_url + "?appid={appid}&currency={currency}&market_hash_name=" + quote(market_hash_name)
    resp = requests.get(url)
    data = resp.json()

    if data.get("success"):
        return {
            "lowest_price": data.get("lowest_price"),
            "median_price": data.get("median_price"),
            "volume": data.get("volume")
        }
    else:
        return None

In [21]:
for item in items:
    name = item["market_hash_name"]
    price_info = get_market_price(name)
    
    if price_info:
        print(f"{name}: {price_info['lowest_price']} (median: {price_info['median_price']})")
    else:
        print(f"{name}: Price not found")

ConnectTimeout: HTTPSConnectionPool(host='steamcommunity.com', port=443): Max retries exceeded with url: /market/priceoverview/?appid=%7Bappid%7D&currency=%7Bcurrency%7D&market_hash_name=Sticker%20%7C%20NRG%20%7C%20Austin%202025 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x75f80e7bf1a0>, 'Connection to steamcommunity.com timed out. (connect timeout=None)'))